![Fraud detection image](cover_image.jpg)

🏦 Banks are battling frauds with machine learning models, but changing data patterns can weaken these defenses. London's Poundbank needs your help to figure out why their fraud detection models aren't as accurate anymore.

Poundbank recommends the `nannyml` library for monitoring machine learning models, which is also their tool of choice.

## The data

They have provided you with a reference(test data) and analysis set(production data). A summary and preview are provided below.

## reference.csv and analysis.csv

| Column     | Description              |
|------------|--------------------------|
| `'timestamp'` | Date of the transaction. |
| `'time_since_login_min'` | Time since the user logged in to the app. |
| `'transaction_amount'` | The amount of Pounds(£) that users sent to another account. |
| `'transaction_type'` | Transaction type: <ul><li>`CASH-OUT` - Withdrawing money from an account.</li><li>`PAYMENT` - Transaction where a payment is made to a third party.</li><li>`CASH-IN` - This is the opposite of a cash-out. It involves depositing money into an account.</li><li>`TRANSFER` - Transaction which involves moving funds from one account to another.</li> |
| `'is_first_transaction'` | A binary indicator denoting if the transaction is the user's first (1 for the first transaction, 0 otherwise). |
| `'user_tenure_months'` | The duration in months since the user's account was created or since they became a member. |
| `'is_fraud'` | A binary label indicating whether the transaction is fraudulent (1 for fraud, 0 otherwise). |
| `'predicted_fraud_proba'` | The probability assigned by a detection model indicates the likelihood of a fraudulent transaction. |
| `'predicted_fraud'` |  The predicted classification label is calculated based on predicted fraud probability by the detection model (1 for predicted fraud, 0 otherwise). |

Questions:
1. Identify the months in which the estimated(expected) and realized(actual) accuracy of the model triggers alerts. 
2. Determine the feature that shows the most drift between the reference and analysis sets, thereby impacting the drop in realized accuracy the most.
3. Figure out why the accuracy dropped. Think of a possible explanation.

In [2]:
# Re-run this cell to install nannyml
!pip install nannyml

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 106.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 161.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 120.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.8/865.8 kB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 157.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 175.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this w

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.40 requires botocore==1.31.40, but you have botocore 1.35.23 which is incompatible.
awswrangler 2.16.1 requires pyarrow<7.1.0,>=2.0.0, but you have pyarrow 14.0.2 which is incompatible.
boto3 1.28.40 requires botocore<1.32.0,>=1.31.40, but you have botocore 1.35.23 which

In [43]:
import pandas as pd
import nannyml as nml
nml.disable_usage_logging()

reference = pd.read_csv("reference.csv")
analysis = pd.read_csv("analysis.csv")
reference.head()

,timestamp,time_since_login_min,transaction_amount,transaction_type,is_first_transaction,user_tenure_months,is_fraud,predicted_fraud_proba,predicted_fraud
0,2018-01-01 00:00:00.000,1.561750,3981.1,PAYMENT,False,0.318980,1.0,0.99,1
1,2018-01-01 00:08:43.152,1.658074,1267.9,PAYMENT,False,7.391323,0.0,0.07,0
2,2018-01-01 00:17:26.304,2.454287,1984.7,CASH-IN,False,0.781225,1.0,1.00,1
3,2018-01-01 00:26:09.456,2.392085,2265.2,CASH-OUT,False,0.680473,1.0,0.98,1
4,2018-01-01 00:34:52.608,2.189806,2126.8,CASH-IN,False,8.542895,1.0,0.99,1


In [4]:
analysis.head()

,timestamp,time_since_login_min,transaction_amount,transaction_type,is_first_transaction,user_tenure_months,predicted_fraud_proba,predicted_fraud,is_fraud
0,2018-11-01 00:04:52.464,2.174243,2832.3,CASH-OUT,False,1.013445,0.97,1,1
1,2018-11-01 00:13:35.616,2.493543,1426.9,CASH-OUT,False,6.700041,0.09,0,0
2,2018-11-01 00:22:18.768,1.807432,1302.0,PAYMENT,False,6.291723,0.01,0,0
3,2018-11-01 00:31:01.920,2.133415,1432.1,PAYMENT,True,8.165503,0.00,0,0
4,2018-11-01 00:39:45.072,1.987827,1870.3,CASH-OUT,False,8.205203,0.03,0,0


In [44]:
analysis.columns

Index(['timestamp', 'time_since_login_min', 'transaction_amount',
       'transaction_type', 'is_first_transaction', 'user_tenure_months',
       'predicted_fraud_proba', 'predicted_fraud', 'is_fraud'],
      dtype='object')

In [6]:
#analysis = analysis.set_index('timestamp')
#reference = reference.set_index('timestamp')

In [45]:
import nannyml as nml

#using CBPE (Chunk-Based Performance Estimation)

cbpe = nml.CBPE(
    y_pred_proba='predicted_fraud_proba',
    y_pred='predicted_fraud',
    y_true='is_fraud',
    problem_type='classification_binary',
    metrics=['accuracy'],
    chunk_period="m",
    timestamp_column_name = 'timestamp'
)

cbpe.fit(reference)

results = cbpe.estimate(analysis)

performance_metrics = results.filter(period='analysis').to_df()

# Convert the results into a DataFrame, which includes alert information
performance_alerts_df = results.to_df()

print(performance_metrics)
print(performance_alerts_df.columns)
performance_alerts_df

     chunk                          ...        accuracy                       
       key chunk_index start_index  ... upper_threshold lower_threshold  alert
0  2018-11           0           0  ...        0.950808        0.936367  False
1  2018-12           1        4955  ...        0.950808        0.936367  False
2  2019-01           2       10074  ...        0.950808        0.936367  False
3  2019-02           3       15194  ...        0.950808        0.936367  False
4  2019-03           4       19818  ...        0.950808        0.936367  False
5  2019-04           5       24938  ...        0.950808        0.936367   True
6  2019-05           6       29893  ...        0.950808        0.936367   True
7  2019-06           7       35012  ...        0.950808        0.936367   True

[8 rows x 15 columns]
MultiIndex([(   'chunk',                       'key'),
            (   'chunk',               'chunk_index'),
            (   'chunk',               'start_index'),
            (   'chunk

chunk                          ...        accuracy                       
        key chunk_index start_index  ... upper_threshold lower_threshold  alert
0   2018-01           0           0  ...        0.950808        0.936367  False
1   2018-02           1        5120  ...        0.950808        0.936367  False
2   2018-03           2        9745  ...        0.950808        0.936367  False
3   2018-04           3       14864  ...        0.950808        0.936367  False
4   2018-05           4       19819  ...        0.950808        0.936367  False
5   2018-06           5       24939  ...        0.950808        0.936367  False
6   2018-07           6       29893  ...        0.950808        0.936367  False
7   2018-08           7       35013  ...        0.950808        0.936367  False
8   2018-09           8       40133  ...        0.950808        0.936367  False
9   2018-10           9       45087  ...        0.950808        0.936367  False
10  2018-11           0           0  ...        0.950808        0.936367  False
11  2018-12           1        4955  ...        0.950808        0.936367  False
12  2019-01           2       10074  ...        0.950808        0.936367  False
13  2019-02           3       15194  ...        0.950808        0.936367  False
14  2019-03           4       19818  ...        0.950808        0.936367  False
15  2019-04           5       24938  ...        0.950808        0.936367   True
16  2019-05           6       29893  ...        0.950808        0.936367   True
17  2019-06           7       35012  ...        0.950808        0.936367   True

[18 rows x 15 columns]

In [48]:
# convert 'start_date' column to a datetime format
performance_alerts_df[('chunk', 'start_date')] = pd.to_datetime(performance_alerts_df[('chunk', 'start_date')])

# filter rows where alerts were triggered for drop in accuracy
alert_rows = performance_alerts_df[performance_alerts_df[('accuracy', 'alert')] == True]

# extract the months from chunk start dates where performance alerts were triggered
alert_timestamps = alert_rows[('chunk', 'start_date')].dt.to_period('M').unique()
months_with_performance_alerts = [f"{month.strftime('%B').lower()}_{month.year}" for month in alert_timestamps]

print("Months with performance alerts:", months_with_performance_alerts)

Months with performance alerts: ['april_2019', 'may_2019', 'june_2019']


['april_2019', 'may_2019', 'june_2019'] are the months with performance alerts


To determine identify the feature shows performance drift between the reference and analysis sets, then identify months when the feature experienced a drift. Lastly, i will try to explain why the performance drifted using the analysis.

I will use Kolmogorov-Smirnov and Chi-square test


In [25]:
analysis.head()

,time_since_login_min,transaction_amount,transaction_type,is_first_transaction,user_tenure_months,predicted_fraud_proba,predicted_fraud,is_fraud
timestamp,,,,,,,,
2018-11-01 00:04:52.464,2.174243,2832.3,CASH-OUT,False,1.013445,0.97,1,1
2018-11-01 00:13:35.616,2.493543,1426.9,CASH-OUT,False,6.700041,0.09,0,0
2018-11-01 00:22:18.768,1.807432,1302.0,PAYMENT,False,6.291723,0.01,0,0
2018-11-01 00:31:01.920,2.133415,1432.1,PAYMENT,True,8.165503,0.00,0,0
2018-11-01 00:39:45.072,1.987827,1870.3,CASH-OUT,False,8.205203,0.03,0,0


In [30]:
reference.head()

,time_since_login_min,transaction_amount,transaction_type,is_first_transaction,user_tenure_months,is_fraud,predicted_fraud_proba,predicted_fraud
timestamp,,,,,,,,
2018-01-01 00:00:00.000,1.561750,3981.1,PAYMENT,False,0.318980,1.0,0.99,1
2018-01-01 00:08:43.152,1.658074,1267.9,PAYMENT,False,7.391323,0.0,0.07,0
2018-01-01 00:17:26.304,2.454287,1984.7,CASH-IN,False,0.781225,1.0,1.00,1
2018-01-01 00:26:09.456,2.392085,2265.2,CASH-OUT,False,0.680473,1.0,0.98,1
2018-01-01 00:34:52.608,2.189806,2126.8,CASH-IN,False,8.542895,1.0,0.99,1


In [57]:
features = ['time_since_login_min', 'transaction_amount', 'user_tenure_months', 
             'transaction_type', 'is_first_transaction']

drift_calculator = nml.UnivariateDriftCalculator(
    timestamp_column_name='timestamp',        
    column_names=features,                    
    chunk_period='m'                        
)

# Set the methods for continuous and categorical features
drift_calculator.continuous_methods = ['kolmogorov_smirnov']
drift_calculator.categorical_methods = ['chi-2']

drift_calculator.fit(reference)

drift_results = drift_calculator.calculate(analysis)

drift_metrics = drift_results.filter(period='analysis').to_df()
drift_metrics

chunk                          ... transaction_type                       
     chunk                          ...   jensen_shannon                       
       key chunk_index start_index  ...  upper_threshold lower_threshold  alert
0  2018-11           0           0  ...         0.022142            None  False
1  2018-12           1        4955  ...         0.022142            None  False
2  2019-01           2       10074  ...         0.022142            None  False
3  2019-02           3       15194  ...         0.022142            None  False
4  2019-03           4       19818  ...         0.022142            None  False
5  2019-04           5       24938  ...         0.022142            None  False
6  2019-05           6       29893  ...         0.022142            None  False
7  2019-06           7       35012  ...         0.022142            None  False

[8 rows x 27 columns]

In [58]:
drift_metrics.columns

MultiIndex([(               'chunk',          'chunk',             'key'),
            (               'chunk',          'chunk',     'chunk_index'),
            (               'chunk',          'chunk',     'start_index'),
            (               'chunk',          'chunk',       'end_index'),
            (               'chunk',          'chunk',      'start_date'),
            (               'chunk',          'chunk',        'end_date'),
            (               'chunk',          'chunk',          'period'),
            ('time_since_login_min', 'jensen_shannon',           'value'),
            ('time_since_login_min', 'jensen_shannon', 'upper_threshold'),
            ('time_since_login_min', 'jensen_shannon', 'lower_threshold'),
            ('time_since_login_min', 'jensen_shannon',           'alert'),
            (  'transaction_amount', 'jensen_shannon',           'value'),
            (  'transaction_amount', 'jensen_shannon', 'upper_threshold'),
            (  'transacti

In [66]:
# extract features that have triggered an alert from the drift metric table above
alerted_features = []
for feature in ['time_since_login_min', 'transaction_amount', 'user_tenure_months', 
                'is_first_transaction', 'transaction_type']:
    if drift_metrics[(feature, 'jensen_shannon', 'alert')].any():
        alerted_features.append(feature)

max_drift_feature = None
max_drift_value = -float('inf')

for feature in alerted_features:
    drift_value = drift_metrics[(feature, 'jensen_shannon', 'value')].max()
    if drift_value > max_drift_value:
        max_drift_value = drift_value
        max_drift_feature = feature

print("Feature with the most drift:", max_drift_feature)


Feature with the most drift: time_since_login_min


In [65]:
alerted_features

['time_since_login_min', 'transaction_amount', 'is_first_transaction']

In [71]:
import matplotlib.pyplot as plt

#visualizing feature drift
fig = drift_results.plot(kind='drift', period='analysis')
fig

The above plot can be used to determine months where the features had a drift

Analyzing the univariate drift metrics suggests that the model expereinenced a performance drift or a drop in performance in the analysis period due to an increase in period between successive user logins ('time_since_login_min'). The increase between successive logins means that this feature's distribution changed substantially over time. Therefore, this could affect how the model distinguishes between normal and fraudulent activities, especially since the model was trained on data with shorter login interval.

If 'time_since_login_min' is contributing significantly to the models performance, the change in distribution could lead to a higher rate of misclassification.